In [6]:
from IPython.display import display, Image, Audio

import cv2  # We're using OpenCV to read video, to install !pip install opencv-python
import base64
import time
# from openai import OpenAI
import os
import requests
import ollama
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [4]:
video = cv2.VideoCapture("How Will Generative AI Shape the Future of Marketing_.mp4")

base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()
print(len(base64Frames), "frames read.")


3067 frames read.


In [7]:
def load_llm():
    """Load the Language Model."""
    llm = Ollama(
        model="llava3",
        verbose=True,
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    )
    
    return llm

In [19]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video that I want to upload. Generate a compelling description that I can upload along with the video.",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
        ],
    },
]
params = {
    "model": load_llm(),
    "messages": PROMPT_MESSAGES,
    "max_tokens": 200,
}

response = ollama.chat(model='llava-phi3')
print(response['message']['content'])